# Apriori算法

- Apriori是一种常用的数据关联规则挖掘方法，它可以用来找出数据集中频繁出现的数据集合。找出这样的一些频繁集合有利于决策，例如通过找出超市购物车数据的频繁项集，可以更好地设计货架的摆放。需要注意的是它是一种逐层迭代的方法，先找出频繁1项集L1，再利用L1找出频繁2项集，以此类推……

- 关联规则：关联规则是形如 X→Y 的蕴涵表达式，其中X和Y是不相交的项集，即 X∩Y=∅。关联规则的强度可以用它的支持度（support）和置信度（confidence）来度量。

支持度：$$support(x,y)=p(x,y)=\frac{num(xy)}{num(allsamples)}$$

置信度：$$confidence(xy)=p(x|y)=\frac{p(xy)}{p(y)}$$

- 最小支持度：最小支持度就是人为规定的阈值，表示项集在统计意义上的最低重要性。 最小置信度：最小置信度也是人为规定的阈值，表示关联规则最低可靠性。 只有支持度与置信度同时达到了最小支持度与最小置信度，此关联规则才会被称为强规则。   
  频繁项集：满足最小支持度的所有项集，称作频繁项集。   
  频繁项集性质：1、频繁项集的所有非空子集也为频繁项集；2、若A项集不是频繁项集，则其他项集或事务与A项集的并集也不是频繁项集）

### 算法流程

输入：数据集合D，支持度阈值，置信度阈值

输出：最大的频繁k项集
1）扫描整个数据集，得到所有出现过的数据，作为候选频繁1项集。

2）挖掘频繁k项集

a) 扫描数据计算候选频繁k项集的支持度

b) 去除候选频繁k项集中支持度低于阈值的数据集,得到频繁k项集。如果得到的频繁k项集为空，则直接返回频繁k-1项集的集合作为算法结果，算法结束。如果得到的频繁k项集只有一项，则直接返回频繁k项集的集合作为算法结果，算法结束。

c) 基于频繁k项集，连接生成候选频繁k+1项集。

3） 令k=k+1，转入步骤2。

从算法的步骤可以看出，Aprior算法每轮迭代都要扫描数据集，因此在数据集很大，数据种类很多的时候，算法效率很低。

缺点:效率低、占用的内存大

In [3]:
#数据集
def load_data_set():
    """
    将题目中的每个列表写入数据集，返回返回数据集
    """
    data_set = [['l1', 'l2'], ['l1', 'l3', 'l4', 'l5'], ['l2', 'l3', 'l4', 'l6'],
            ['l1', 'l2', 'l3', 'l4'], ['l1', 'l2', 'l3','l6']]
    return data_set


def create_C1(data_set):
    """
    通过扫描数据集创建频繁候选1项集C1。
    返回：
    C1：包含所有频繁候选1项集的集合
    """
    C1 = set()  # 创建一个空集合，无序不重合
    for t in data_set:
        for item in t:
            item_set = frozenset([item])    #返回一个不可变集合
            C1.add(item_set)    #加入到第一项集
    return C1


def is_apriori(Ck_item, Lksub1):
    """
    判断频繁候选k项集是否满足先验性质。子集是否为频繁项集
    参数：
    Ck_item：Ck中包含所有频繁项的频繁候选k项集
    Lksub1:Lk-1，包含所有频繁候选（k-1）项集的集合。
    返回：
    真：满足先验属性。
    False：不满足Apriori属性。
    """
    for item in Ck_item:
        # 判断子集是否在频繁项集里面，不在返回flase，在返回ture
        sub_Ck = Ck_item - frozenset([item])
        if sub_Ck not in Lksub1:
            return False
    return True


def create_Ck(Lksub1, k):
    """
    创建Ck，一个包含所有频繁候选k项集的集合
    根据k-1的频繁项集连接求出k项集
    参数：
    Lksub1:包含所有频繁候选（k-1）项集的集合。
    len_Lksub1：k-1频繁项集的长度
    返回：
    Ck：包含所有频繁候选k项集的集合。
    """
    Ck = set()
    len_Lksub1 = len(Lksub1)
    list_Lksub1 = list(Lksub1)# 集合转换成列表
    for i in range(len_Lksub1): #对k-1频繁项集进行连接
        for j in range(1, len_Lksub1):
            l1 = list(list_Lksub1[i])
            l2 = list(list_Lksub1[j])
            l1.sort()   #都进行排序，对前k-2项进行比较
            l2.sort()
            if l1[0:k-2] == l2[0:k-2]:  #前k-2项相等则进行连接
                Ck_item = list_Lksub1[i] | list_Lksub1[j]
                if is_apriori(Ck_item, Lksub1): #判断子集是否为频繁项集，是则加入到第k项集
                    Ck.add(Ck_item)
    return Ck


def generate_Lk_by_Ck(data_set, Ck, min_support, support_data):
    """
    通过从Ck执行delete策略生成Lk。
    参数
    Ck：第k项集的集合。
    min_support：最小支持度。
    item_count:统计ck中每个候选项集出现的次数
    support——data：字典。键是frequency itemset(项集)，值是support(支持度)。
    t_num:数据集合里面列表的个数
    返回：
    Lk：包含所有频繁k项集的集合。
    """
    Lk = set()
    item_count = {}#空字典
    for t in data_set:  #项集里的每个候选频繁项集都要看是否为 数据集里面的每个列表的子集，若是则出现的次数加一
        for item in Ck:
            if item.issubset(t):    #
                if item not in item_count:
                    item_count[item] = 1#不在字典中则添加进去，初值为1
                else:
                    item_count[item] += 1   #已经存在，数值加一
    t_num = float(len(data_set))    #求数据集里面多少个列表
    # 通过for循环选出满足最小支持度的频繁项集，并添加到support_data
    for item in item_count:
        if (item_count[item] / t_num) >= min_support:
            Lk.add(item)
            support_data[item] = item_count[item] / t_num
    return Lk


def generate_L(data_set, min_support):
    """
    生成所有频繁项集。
    参数：
    data_set：数据集。
    k： 所有频繁项集的最大项数。
    min_support：最小支持度。
    support_data：字典。键是frequency itemset(频繁项集)，值是support(频繁项集的支持度)。存放频繁所有频繁项集以及它的支持度
    c1：所有第一项集
    L1:第一频繁项集
    LKsub:存放第n项集，然后添加到频繁项集列表L
    返回：
    L： 频繁项集列表。
    """
    support_data = {}   #空字典
    C1 = create_C1(data_set)# 产生所有的第一项集
    L1 = generate_Lk_by_Ck(data_set, C1, min_support, support_data)#求第一频繁项集
    Lksub1 = L1.copy()  #  Lksub1复制第一频繁项集给频繁项集L
    L = []
    L.append(Lksub1)    # 添加第一频繁项集
    i=2 #第一频繁项集已经求了，根据求出的第一频繁项集求第二项集...频繁项集的子集都是频繁项集
    while(1):
      Ci = create_Ck(Lksub1, i) #产生第i项集
      Li = generate_Lk_by_Ck(data_set, Ci, min_support, support_data)   #产生第i频繁项集
      if len(Li)==0:    #当li为空时，更大的也就没有了，调出循环
        break
      Lksub1 = Li.copy()
      L.append(Lksub1)
      i+=1
    return L, support_data


def generate_big_rules(L, support_data, min_conf):
    """
    找出满足最小置信度的频繁项集
    参数：
    L：频繁项集。
    support_data：字典。键是frequency itemset（某频繁项集），值是support（支持度）。频繁项集以及每个项集的支持度
    min_conf：最置信度。
    返回
    满足最小值置信度的频繁项集(强关联规则)
    """
    big_rule_list = []  # 强关联规则的列表
    sub_set_list = []   # 子集列表
    for i in range(0, len(L)):  # 对于每个频繁项集要产生它的所有非空子集，计算他的置信度，大于最小置信度，则将子集，以及剩下的一些项还有置信度添加到强关联的列表里面
        for freq_set in L[i]:   #遍历n频繁项集，第一频繁项集里面没有强关联
            for sub_set in sub_set_list:    #频繁项集的子集也一定是频繁项集
                if sub_set.issubset(freq_set):  # 判断sub_set是频繁项集的子集
                    conf = support_data[freq_set] / support_data[freq_set - sub_set]    #计算置信度
                    big_rule = (freq_set - sub_set, sub_set, conf)  #将关联规则以及置信度放入big_rule，如果大于最小置信度则加到强关联规则的列表里面
                    if conf >= min_conf and big_rule not in big_rule_list:  #满足最小置信度并之前没有添加
                        big_rule_list.append(big_rule)
            sub_set_list.append(freq_set)
    return big_rule_list


if __name__ == "__main__":
    data_set = load_data_set() # 加载数据集
    L, support_data = generate_L(data_set, min_support=0.6) # 生成所有频繁项集L，以及含有频繁项集的支持度的字典support_data
    big_rules_list = generate_big_rules(L, support_data, min_conf=0.7)  #找出满足最小置信度的频繁项集强关联规则
    # 输出频繁项集
    for Lk in L:
        print("="*50)
        print("frequent " + str(len(list(Lk)[0])) + "-itemsets\t\tsupport")
        print("="*50)
        for freq_set in Lk:
            print(freq_set, support_data[freq_set])
    print("Big Rules")
    # 输出强关联规则
    for item in big_rules_list:
        print(item[0], "=>", item[1], "conf: ", item[2])


frequent 1-itemsets		support
frozenset({'l2'}) 0.8
frozenset({'l3'}) 0.8
frozenset({'l4'}) 0.6
frozenset({'l1'}) 0.8
frequent 2-itemsets		support
frozenset({'l3', 'l1'}) 0.6
frozenset({'l2', 'l3'}) 0.6
frozenset({'l2', 'l1'}) 0.6
frozenset({'l3', 'l4'}) 0.6
Big Rules
frozenset({'l1'}) => frozenset({'l3'}) conf:  0.7499999999999999
frozenset({'l3'}) => frozenset({'l1'}) conf:  0.7499999999999999
frozenset({'l3'}) => frozenset({'l2'}) conf:  0.7499999999999999
frozenset({'l2'}) => frozenset({'l3'}) conf:  0.7499999999999999
frozenset({'l1'}) => frozenset({'l2'}) conf:  0.7499999999999999
frozenset({'l2'}) => frozenset({'l1'}) conf:  0.7499999999999999
frozenset({'l4'}) => frozenset({'l3'}) conf:  1.0
frozenset({'l3'}) => frozenset({'l4'}) conf:  0.7499999999999999
